In [8]:
import os
import glob
from glob import glob as gg

from collections import defaultdict

import re
import sqlite3

import pandas as pd
from pandas import DataFrame
from pandas import Series

import numpy as np
from numpy import random, asarray, linspace, corrcoef
from datetime import datetime, date, time

import jieba
import jieba.posseg as pseg
import jieba.analyse
from optparse import OptionParser
from __future__ import print_function, unicode_literals
import sys

get_ipython().magic(u'pwd')
get_ipython().magic(u'cd "/Users/weiaiwaynexu/Dropbox/Acer Laptop Sync/Data Science/OC"')

con = sqlite3.connect("OccupyHK.sqlite")
cur = con.cursor()
df = pd.read_sql_query("SELECT * FROM Posts", con)
con.close()
df['published_date'] = pd.to_datetime(df['published_date'])
df = df.sort_values(by='published_date')
df['share_count'] = pd.to_numeric(df['share_count'])
df = df.fillna(0)
df = df[(df['published_date'] >= '2014-09-01') & (df['published_date'] <= "2014-12-30")]
print ("the earliest date:", df['published_date'].head(1), "the latest date:", df['published_date'].tail(1))

yellow_official = df.loc[df['org_name'].isin(['學民思潮 Scholarism',
                                                 'Occupy Central 佔領中環', 
                                                 '讓愛與和平佔領中環 Occupy Central with Love and Peace', 
                                                 '926 平民在政總現場','佔領期間 民間智慧選輯', '傘下爸媽',
                                                 '香港天主教正義和平委員會','Hong Kong Democracy Now', '踢爆五毛','香港警權關注組',
                                                 '維護香港核心價值', '我係香港人', '香港人 Secrets','保衛香港自由聯盟','9反佔中 HK oic',
                                                 'Fluid Occupiers'])] 


blue_official = df.loc[df['org_name'].isin(['幫港出聲 Silent Majority','Speak Out HK 港人講地','藍絲帶運動 The Blue Ribbon Movement','向香港警察致敬',
                                               '反佔中苦主大聯盟','泛民不代表我','聲討黃之鋒',
                                               '一人一Like救學生，反對中學生衝擊警察及佔中！',
                                               '「保普選 反佔中」簽左名揮手區！',
                                               '我要真普洱行動','我撐警務處長曾偉雄！',
                                               '我係大學生，我愛國愛港'])] 


print ("There are",len(yellow_official), "OFFICIAL posts sent by anti-establishment civil groups")
print ("There are",len(blue_official), "OFFICIAL posts sent by pro-establishment civil groups")

/Users/weiaiwaynexu/Dropbox/Acer Laptop Sync/Data Science/OC
the earliest date: 8635   2014-09-01
Name: published_date, dtype: datetime64[ns] the latest date: 34084   2014-12-24 23:00:01
Name: published_date, dtype: datetime64[ns]
There are 23884 OFFICIAL posts sent by anti-establishment civil groups
There are 6477 OFFICIAL posts sent by pro-establishment civil groups


In [55]:
yellow_official_top = yellow_official[yellow_official['share_count'] >= yellow_official["share_count"].quantile(0.75)]
yellow_official_bottom = yellow_official[yellow_official['share_count'] == 0]
blue_official_top = blue_official[blue_official['share_count'] >= blue_official["share_count"].quantile(0.75)]
blue_official_bottom = blue_official[blue_official['share_count'] == 0]

In [56]:
def convert2corpus(txt):
    corpus = []
    for index, i in txt.iterrows():
        corpus.append(i['content'])
    corpus = str(corpus)
    corpus = corpus.replace("[" and "]" and "\\", " ")
    return corpus

def cnpuncttoen(corpus):
    #corpus_new = corpus.replace(".", "")
    corpus_new = corpus.replace("。", "*.")
    corpus_new = corpus_new.replace("？", "*?")
    corpus_new = corpus_new.replace(",", "*,")
    corpus_new = corpus_new.replace("！", "*!")
    return corpus_new

def sentence_split(str_centence):
    list_ret = list()
    for s_str in str_centence.split('*.'):
        if '*?' in s_str:
            list_ret.extend(s_str.split('*?'))
        elif '*!' in s_str:
            list_ret.extend(s_str.split('*!'))
        else:
            list_ret.append(s_str)
    return list_ret

yellow_official_top_corpus = sentence_split(cnpuncttoen(convert2corpus(yellow_official_top))) 
yellow_official_bottom_corpus = sentence_split(cnpuncttoen(convert2corpus(yellow_official_bottom))) 
blue_official_top_corpus = sentence_split(cnpuncttoen(convert2corpus(blue_official_top)))
blue_official_bottom_corpus = sentence_split(cnpuncttoen(convert2corpus(blue_official_bottom))) 

In [57]:
jieba.analyse.set_stop_words('cn_stopword.txt')

def cracut(txt):
    cuttxt1 = [[(j.word, j.flag) for j in jieba.posseg.cut(i)] for i in txt]
    cracut = [[token for token, tag in sent if re.match('n|a', tag) and len(token)>1] 
                for sent in cuttxt1]
    return cracut

def cut(txt):
    cuttxt2 = [[(j.word, j.flag) for j in jieba.posseg.cut(i)] for i in txt]
    cut = [[token for token, tag in sent if len(token)>1] 
                for sent in cuttxt2]
    return cut

yellow_official_top_cra = cracut(yellow_official_top_corpus)
yellow_official_bottom_cra = cracut(yellow_official_bottom_corpus)
blue_official_top_cra = cracut(blue_official_top_corpus)
blue_official_bottom_cra = cracut(blue_official_bottom_corpus)

yellow_official_top_cut = cut(yellow_official_top_corpus)
yellow_official_bottom_cut = cut(yellow_official_bottom_corpus)
blue_official_top_cut = cut(blue_official_top_corpus)
blue_official_bottom_cut = cut(blue_official_bottom_corpus)

In [58]:
import itertools as it 
import networkx as nx

def semanticnetwork(inputlist):
    edgelist = [edge for phrase in inputlist for edge in it.combinations(phrase, 2)]
    word_network = nx.Graph(edgelist)
    return word_network

yellow_official_top_nx = semanticnetwork(yellow_official_top_cra)
yellow_official_bottom_nx = semanticnetwork(yellow_official_bottom_cra)
blue_official_top_nx = semanticnetwork(blue_official_top_cra)
blue_official_bottom_nx = semanticnetwork(blue_official_bottom_cra)

In [59]:
nx.write_gpickle(yellow_official_top_nx, "ARNOVA Analysis/yellow_official_top_nx_during.pickle")
nx.write_gpickle(yellow_official_bottom_nx, "ARNOVA Analysis/yellow_official_bottom_nx_during.pickle")
nx.write_gpickle(blue_official_top_nx, "ARNOVA Analysis/blue_official_top_nx_during.pickle")
nx.write_gpickle(blue_official_bottom_nx, "ARNOVA Analysis/blue_official_bottom_nx_during.pickle")
#nx.write_graphml(xxxx, "ARNOVA Analysis/test_cra_nx.graphml")


In [60]:
yellow_official_top_bt = nx.betweenness_centrality(yellow_official_top_nx, normalized=True)
yellow_official_top_bt = sorted(yellow_official_top_bt.items(), key=lambda x:x[1], reverse=True)
yellow_official_top_bt = DataFrame(data=yellow_official_top_bt)
yellow_official_top_bt.to_csv("ARNOVA Analysis/yellow_official_top_bt_during.csv", sep=',')

yellow_official_bottom_bt = nx.betweenness_centrality(yellow_official_bottom_nx, normalized=True)
yellow_official_bottom_bt = sorted(yellow_official_bottom_bt.items(), key=lambda x:x[1], reverse=True)
yellow_official_bottom_bt = DataFrame(data=yellow_official_bottom_bt)
yellow_official_bottom_b t.to_csv("ARNOVA Analysis/yellow_official_bottom_bt_during.csv", sep=',')

blue_official_top_bt = nx.betweenness_centrality(blue_official_top_nx, normalized=True)
blue_official_top_bt = sorted(blue_official_top_bt.items(), key=lambda x:x[1], reverse=True)
blue_official_top_bt = DataFrame(data=blue_official_top_bt)
blue_official_top_bt.to_csv("ARNOVA Analysis/blue_official_top_bt_during.csv", sep=',')

blue_official_bottom_bt = nx.betweenness_centrality(blue_official_bottom_nx, normalized=True)
blue_official_bottom_bt = sorted(blue_official_bottom_bt.items(), key=lambda x:x[1], reverse=True)
blue_official_bottom_bt = DataFrame(data=blue_official_bottom_bt)
blue_official_bottom_bt.to_csv("ARNOVA Analysis/blue_official_bottom _bt_during.csv", sep=',')

In [86]:
import gensim
from gensim import corpora

dictionary = corpora.Dictionary(yellow_official_top_cra)
dictionary.save('ARNOVA Analysis/yellow_official_top_cra.dict') 
yellow_official_top_cra_TM = [dictionary.doc2bow(doc) for doc in yellow_official_top_cra]
corpora.MmCorpus.serialize('ARNOVA Analysis/yellow_official_top_cra_during.mm', yellow_official_top_cra_TM)

dictionary = corpora.Dictionary(yellow_official_bottom_cra)
dictionary.save('ARNOVA Analysis/yellow_official_bottom_cra.dict') 
yellow_official_bottom_cra_TM = [dictionary.doc2bow(doc) for doc in yellow_official_bottom_cra]
corpora.MmCorpus.serialize('ARNOVA Analysis/yellow_official_bottom_cra_during.mm', yellow_official_bottom_cra_TM)

dictionary = corpora.Dictionary(blue_official_top_cra)
dictionary.save('ARNOVA Analysis/blue_official_top_cra.dict') 
blue_official_top_cra_TM = [dictionary.doc2bow(doc) for doc in blue_official_top_cra]
corpora.MmCorpus.serialize('ARNOVA Analysis/blue_official_top_cra_during.mm', blue_official_top_cra_TM)

dictionary = corpora.Dictionary(blue_official_bottom_cra)
dictionary.save('ARNOVA Analysis/blue_official_bottom_cra.dict') 
blue_official_bottom_cra_TM = [dictionary.doc2bow(doc) for doc in blue_official_bottom_cra]
corpora.MmCorpus.serialize('ARNOVA Analysis/blue_official_bottom_cra_during.mm', blue_official_bottom_cra_TM)


In [95]:
dictionary = corpora.Dictionary(yellow_official_top_cut)
dictionary.save('ARNOVA Analysis/yellow_official_top_cut.dict') 
yellow_official_top_cut_TM = [dictionary.doc2bow(doc) for doc in yellow_official_top_cut]
corpora.MmCorpus.serialize('ARNOVA Analysis/yellow_official_top_cut_during.mm', yellow_official_top_cut_TM)

dictionary = corpora.Dictionary(yellow_official_bottom_cut)
dictionary.save('ARNOVA Analysis/yellow_official_bottom_cut.dict') 
yellow_official_bottom_cut_TM = [dictionary.doc2bow(doc) for doc in yellow_official_bottom_cut]
corpora.MmCorpus.serialize('ARNOVA Analysis/yellow_official_bottom_cut_during.mm', yellow_official_bottom_cut_TM)

dictionary = corpora.Dictionary(blue_official_top_cut)
dictionary.save('ARNOVA Analysis/blue_official_top_cut.dict') 
blue_official_top_cut_TM = [dictionary.doc2bow(doc) for doc in blue_official_top_cut]
corpora.MmCorpus.serialize('ARNOVA Analysis/blue_official_top_cut_during.mm', blue_official_top_cut_TM)

dictionary = corpora.Dictionary(blue_official_bottom_cut)
dictionary.save('ARNOVA Analysis/blue_official_bottom_cut.dict') 
blue_official_bottom_cut_TM = [dictionary.doc2bow(doc) for doc in blue_official_bottom_cut]
corpora.MmCorpus.serialize('ARNOVA Analysis/blue_official_bottom_cut_during.mm', blue_official_bottom_cut_TM)


In [115]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(yellow_official_top_cra_TM, num_topics=8, id2word = dictionary.load("ARNOVA Analysis/yellow_official_top_cra.dict"), passes=50)

In [98]:
print(ldamodel)

LdaModel(num_terms=19475, num_topics=10, decay=0.5, chunksize=2000)


In [64]:
ldamodel.save('ARNOVA Analysis/model.lda')

In [112]:
print(ldamodel.print_topics(num_topics=8, num_words=5))

[(0, '0.023*現場 + 0.021*人士 + 0.021*學民 + 0.021*市民 + 0.019*思潮'), (1, '0.058*政府 + 0.024*香港 + 0.023*梁振英 + 0.016*市民 + 0.015*民主'), (2, '0.029*旺角 + 0.012*警棍 + 0.011*市民 + 0.009*示威者 + 0.008*胡椒'), (3, '0.061*警方 + 0.030*警員 + 0.025*記者 + 0.025*示威者 + 0.021*警隊'), (4, '0.067*香港 + 0.021*公民 + 0.019*社會 + 0.018*抗命 + 0.018*學生')]


In [116]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, yellow_official_top_cra_TM, dictionary=dictionary.load("ARNOVA Analysis/yellow_official_top_cra.dict"))

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      18.855832        1       1 -0.222664  0.229696
2      16.450019        1       2  0.223727  0.105988
5      15.940498        1       3  0.177614  0.158258
6      11.905341        1       4  0.051001 -0.111838
7      10.327254        1       5 -0.183071  0.022990
4       9.698946        1       6 -0.025614 -0.066632
0       8.612678        1       7  0.114563 -0.153330
1       8.209432        1       8 -0.135556 -0.185132, topic_info=      Category         Freq Term        Total  loglift  logprob
term                                                           
7807   Default  2419.000000   警方  2419.000000  30.0000  30.0000
1894   Default  3577.000000   香港  3577.000000  29.0000  29.0000
2983   Default  1949.000000   政府  1949.000000  28.0000  28.0000
18428  Default  1310.000000   旺角  1310.000000  27.0000  27.0000
10477  Default  1132.000000   公民  1132.000000  26.0000  26.0000
9480   Default  1328.000000   學生  1328.000000  25.0000  25.0000
678    Default  1136.000000  示威者  1136.000000  24.0000  24.0000
14887  Default  1343.000000   人士  1343.000000  23.0000  23.0000
4182   Default   735.000000   抗命   735.000000  22.0000  22.0000
4063   Default   669.000000   學民   669.000000  21.0000  21.0000
11941  Default  1049.000000   警員  1049.000000  20.0000  20.0000
8489   Default   606.000000   思潮   606.000000  19.0000  19.0000
1405   Default  1018.000000   社會  1018.000000  18.0000  18.0000
7407   Default   894.000000   民主   894.000000  17.0000  17.0000
5707   Default   715.000000   清場   715.000000  16.0000  16.0000
4227   Default   859.000000   記者   859.000000  15.0000  15.0000
14105  Default   740.000000   現場   740.000000  14.0000  14.0000
9224   Default   685.000000   警隊   685.000000  13.0000  13.0000
14208  Default   937.000000   政治   937.000000  12.0000  12.0000
12723  Default   381.000000   廣場   381.000000  11.0000  11.0000
18009  Default   505.000000   新聞   505.000000  10.0000  10.0000
10545  Default  1996.000000   市民  1996.000000   9.0000   9.0000
5229   Default   431.000000   法庭   431.000000   8.0000   8.0000
7291   Default   362.000000  黃之鋒   362.000000   7.0000   7.0000
17285  Default   611.000000   金鐘   611.000000   6.0000   6.0000
11027  Default   493.000000   自由   493.000000   5.0000   5.0000
6520   Default   749.000000   和平   749.000000   4.0000   4.0000
2879   Default   562.000000   問題   562.000000   3.0000   3.0000
5561   Default   407.000000   學聯   407.000000   2.0000   2.0000
15334  Default   457.000000   中國   457.000000   1.0000   1.0000
...        ...          ...  ...          ...      ...      ...
8512    Topic8    67.453358   合法    68.225258   2.4885  -5.3101
6484    Topic8    63.830187   雙手    64.601365   2.4879  -5.3654
17451   Topic8    58.131518   健民    58.902093   2.4867  -5.4589
2504    Topic8    57.786357   日本    58.557196   2.4866  -5.4648
16697   Topic8    57.167836   小組    57.938969   2.4865  -5.4756
17495   Topic8    57.088693   熱線    57.859274   2.4865  -5.4770
17768   Topic8    53.855139   法官    54.626721   2.4857  -5.5353
15559   Topic8    53.274035   年齡    54.044569   2.4855  -5.5461
14636   Topic8    51.165457   社民    51.936635   2.4849  -5.5865
16079   Topic8    50.572228   短訊    51.342814   2.4848  -5.5982
17220   Topic8    50.509318   工會    51.280639   2.4847  -5.5994
6927    Topic8    48.204899   林夕    48.975747   2.4840  -5.6461
7578    Topic8    47.463429   團結    48.234330   2.4838  -5.6616
10322   Topic8    45.014777   員工    45.786072   2.4829  -5.7146
12978   Topic8    44.288037   狀況    45.059087   2.4826  -5.7309
10512   Topic8    40.666476  陳家洛    41.437154   2.4811  -5.8162
12333   Topic8    39.900752   終於    40.671891   2.4807  -5.8352
8315    Topic8    39.239361  張超雄    40.010262   2.4804  -5.8519
8037    Topic8    38.083976  金鐘道    38.854769   2.4798  -5.8818
5904    Topic8   151.458467   身份   187.715574   2.2853  -4.5013
103     Topic8    74.0954

In [117]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(blue_official_top_cra_TM, num_topics=8, id2word = dictionary.load("ARNOVA Analysis/blue_official_top_cra.dict"), passes=50)

In [118]:
pyLDAvis.gensim.prepare(ldamodel, blue_official_top_cra_TM, dictionary=dictionary.load("ARNOVA Analysis/blue_official_top_cra.dict"))

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      18.879007        1       1 -0.267479  0.114037
1      17.709088        1       2 -0.115652  0.183610
2      14.929451        1       3  0.215739  0.193671
4      11.299942        1       4 -0.023952 -0.198855
6       9.894947        1       5  0.016004 -0.148053
5       9.561554        1       6 -0.067256 -0.084401
7       9.546704        1       7  0.183572  0.043572
3       8.179308        1       8  0.059023 -0.103581, topic_info=      Category         Freq Term        Total  loglift  logprob
term                                                           
1411   Default  3218.000000   香港  3218.000000  30.0000  30.0000
3848   Default  1216.000000   大家  1216.000000  29.0000  29.0000
5736   Default  1464.000000   警方  1464.000000  28.0000  28.0000
9460   Default  1100.000000  示威者  1100.000000  27.0000  27.0000
7019   Default  1082.000000   學生  1082.000000  26.0000  26.0000
1320   Default   905.000000   港人   905.000000  25.0000  25.0000
7070   Default   674.000000   連結   674.000000  24.0000  24.0000
8847   Default   677.000000   講地   677.000000  23.0000  23.0000
7807   Default   619.000000   原文   619.000000  22.0000  22.0000
8841   Default   765.000000   警員   765.000000  21.0000  21.0000
10526  Default   645.000000   專頁   645.000000  20.0000  20.0000
7838   Default  1242.000000   市民  1242.000000  19.0000  19.0000
13476  Default   648.000000   旺角   648.000000  18.0000  18.0000
10300  Default   992.000000   民主   992.000000  17.0000  17.0000
11294  Default   590.000000   中國   590.000000  16.0000  16.0000
4174   Default   378.000000   清場   378.000000  15.0000  15.0000
10949  Default   891.000000   人士   891.000000  14.0000  14.0000
9391   Default   370.000000   大學   370.000000  13.0000  13.0000
4521   Default   489.000000   公民   489.000000  12.0000  12.0000
13796  Default   336.000000   原圖   336.000000  11.0000  11.0000
4061   Default   447.000000   學聯   447.000000  10.0000  10.0000
8796   Default   604.000000   法治   604.000000   9.0000   9.0000
3088   Default   263.000000   記者   263.000000   8.0000   8.0000
451    Default   381.000000   領者   381.000000   7.0000   7.0000
2195   Default   980.000000   政府   980.000000   6.0000   6.0000
4082   Default   364.000000   經濟   364.000000   5.0000   5.0000
3569   Default   346.000000   議員   346.000000   4.0000   4.0000
1035   Default   997.000000   社會   997.000000   3.0000   3.0000
12678  Default   303.000000   金鐘   303.000000   2.0000   2.0000
13825  Default   382.000000   事件   382.000000   1.0000   1.0000
...        ...          ...  ...          ...      ...      ...
3413    Topic8    25.572376   模式    26.356158   2.4734  -5.9686
14155   Topic8    25.317791   關鍵    26.101778   2.4731  -5.9786
13419   Topic8    23.645617   王丹    24.429177   2.4710  -6.0469
4334    Topic8    23.490750   宗教    24.275306   2.4707  -6.0535
1791    Topic8    23.020646   著名    23.804939   2.4701  -6.0737
2472    Topic8    21.380712   心痛    22.164522   2.4676  -6.1476
4654    Topic8    21.171394   感情    21.955217   2.4672  -6.1575
13297   Topic8    21.012619   上司    21.796307   2.4669  -6.1650
5993    Topic8    20.923030   資本    21.707064   2.4668  -6.1693
4563    Topic8    86.294356   入境    98.258258   2.3737  -4.7524
1025    Topic8    98.507633   時候   132.989301   2.2034  -4.6200
11453   Topic8    92.535372   同學   133.476551   2.1372  -4.6825
13041   Topic8    55.711973   事情    78.626726   2.1590  -5.1899
10084   Topic8   100.687812   筆者   199.120718   1.8217  -4.5981
3068    Topic8    39.559397   鏡頭    51.890464   2.2322  -5.5323
1846    Topic8    95.272401   結果   195.566066   1.7844  -4.6534
7762    Topic8    77.759302   理想   153.978645   1.8204  -4.8565
5696    Topic8    54.365100   中者    87.662710   2.0258  -5.2144
11669   Topic8    44.196272   原因    63.029257   2.1486  -5.4215
11459   Topic8    72.065145   日原   153.002939   1.7507  -4.9326
10727   Topic8    62.1314

In [119]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(yellow_official_bottom_cra_TM, num_topics=8, id2word = dictionary.load("ARNOVA Analysis/yellow_official_bottom_cra.dict"), passes=50)

In [120]:
pyLDAvis.gensim.prepare(ldamodel, yellow_official_bottom_cra_TM, dictionary=dictionary.load("ARNOVA Analysis/yellow_official_bottom_cra.dict"))

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      18.546677        1       1 -0.124158 -0.109051
2      14.990974        1       2 -0.138070  0.109157
5      14.553102        1       3 -0.024716 -0.140720
7      12.639289        1       4 -0.056294  0.149913
0      11.857836        1       5 -0.074916  0.010558
6      10.945590        1       6  0.094362 -0.133946
3       9.103853        1       7  0.121352  0.039132
4       7.362680        1       8  0.202440  0.074957, topic_info=     Category         Freq Term        Total  loglift  logprob
term                                                          
1783  Default  1101.000000   香港  1101.000000  30.0000  30.0000
5274  Default   159.000000   清場   159.000000  29.0000  29.0000
8393  Default   276.000000   警方   276.000000  28.0000  28.0000
3656  Default   103.000000   詳情   103.000000  27.0000  27.0000
3780  Default   105.000000   學聯   105.000000  26.0000  26.0000
1617  Default   102.000000   五毛   102.000000  25.0000  25.0000
6023  Default   144.000000   旺角   144.000000  24.0000  24.0000
4869  Default   275.000000   大家   275.000000  23.0000  23.0000
5238  Default   157.000000   中國   157.000000  22.0000  22.0000
1016  Default   348.000000   市民   348.000000  21.0000  21.0000
4611  Default    70.000000   聲明    70.000000  20.0000  20.0000
2783  Default   402.000000   政府   402.000000  19.0000  19.0000
3692  Default    78.000000   英國    78.000000  18.0000  18.0000
4669  Default    63.000000   雨傘    63.000000  17.0000  17.0000
6246  Default    53.000000   課程    53.000000  16.0000  16.0000
4057  Default   199.000000   民主   199.000000  15.0000  15.0000
4896  Default    61.000000  禁制令    61.000000  14.0000  14.0000
6290  Default   146.000000  梁振英   146.000000  13.0000  13.0000
4176  Default    79.000000   現場    79.000000  12.0000  12.0000
6589  Default    49.000000   專業    49.000000  11.0000  11.0000
3010  Default    74.000000  示威者    74.000000  10.0000  10.0000
1397  Default    76.000000   領區    76.000000   9.0000   9.0000
6948  Default    86.000000   金鐘    86.000000   8.0000   8.0000
4694  Default    49.000000   泛民    49.000000   7.0000   7.0000
4372  Default   111.000000   特首   111.000000   6.0000   6.0000
4888  Default    65.000000   警員    65.000000   5.0000   5.0000
8138  Default    45.000000   事務    45.000000   4.0000   4.0000
1944  Default    69.000000   國家    69.000000   3.0000   3.0000
3412  Default    36.000000   武力    36.000000   2.0000   2.0000
4419  Default    41.000000   老師    41.000000   1.0000   1.0000
...       ...          ...  ...          ...      ...      ...
2460   Topic8     5.414450   港股     6.124391   2.4855  -6.2582
5796   Topic8     5.414450  交易日     6.124391   2.4855  -6.2582
5360   Topic8     5.414255  陳為廷     6.124405   2.4855  -6.2582
6358   Topic8     5.337379   日報     6.047714   2.4838  -6.2725
4376   Topic8     5.316145   熱烈     6.026262   2.4834  -6.2765
4878   Topic8     5.266445   貼士     5.976535   2.4823  -6.2859
2317   Topic8     5.266445   白戶     5.976535   2.4823  -6.2859
8441   Topic8     8.046781   太陽     9.469850   2.4459  -5.8620
5202   Topic8    12.953407   重新    19.511903   2.1991  -5.3859
3027   Topic8     9.629156   畢業    14.684161   2.1868  -5.6825
7322   Topic8    13.522719   教授    25.733032   1.9653  -5.3429
201    Topic8     9.571544  王占陽    15.220867   2.1449  -5.6885
5827   Topic8    10.775011   目標    22.341814   1.8795  -5.5700
904    Topic8     9.295184   現實    17.394766   1.9821  -5.7178
947    Topic8     9.026733   話題    16.790415   1.9881  -5.7471
1965   Topic8     9.791801   顏色    20.870286   1.8520  -5.6657
8611   Topic8    13.851048   蘋果    47.065385   1.3856  -5.3189
4979   Topic8    13.966735   示威    50.473768   1.3240  -5.3106
3079   Topic8     6.713756   整體     9.238525   2.2895  -6.0431
3024   Topic8    10.762398   中學    37.348425   1.3645  -5.5712
8710   Topic8     9.391107   警隊    24.962867   1.6311  -5.7075
2325   Topic8  

In [121]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(blue_official_bottom_cra_TM, num_topics=8, id2word = dictionary.load("ARNOVA Analysis/blue_official_bottom_cra.dict"), passes=50)

In [122]:
pyLDAvis.gensim.prepare(ldamodel, blue_official_bottom_cra_TM, dictionary=dictionary.load("ARNOVA Analysis/blue_official_bottom_cra.dict"))

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      18.290733        1       1 -0.146882  0.031361
5      13.789092        1       2 -0.017576 -0.049982
4      13.452591        1       3  0.042142 -0.152488
0      12.723002        1       4 -0.059599  0.044000
6      12.549410        1       5 -0.019891 -0.050117
1      10.916066        1       6  0.098889  0.059681
7       9.391064        1       7  0.076689  0.055388
3       8.888041        1       8  0.026228  0.062156, topic_info=     Category        Freq  Term       Total  loglift  logprob
term                                                         
30    Default   31.000000    學生   31.000000  30.0000  30.0000
337   Default   51.000000    大家   51.000000  29.0000  29.0000
1354  Default  110.000000    香港  110.000000  28.0000  28.0000
1402  Default   26.000000    民主   26.000000  27.0000  27.0000
890   Default   17.000000    集會   17.000000  26.0000  26.0000
264   Default   19.000000    法治   19.000000  25.0000  25.0000
1600  Default   14.000000   藍絲帶   14.000000  24.0000  24.0000
465   Default   23.000000    警方   23.000000  23.0000  23.0000
1562  Default   18.000000    政治   18.000000  22.0000  22.0000
689   Default   11.000000  香港市民   11.000000  21.0000  21.0000
1535  Default   13.000000    國際   13.000000  20.0000  20.0000
1325  Default    8.000000    記者    8.000000  19.0000  19.0000
671   Default   13.000000    清場   13.000000  18.0000  18.0000
585   Default    6.000000    精神    6.000000  17.0000  17.0000
1426  Default    8.000000    黃絲    8.000000  16.0000  16.0000
859   Default   11.000000    和平   11.000000  15.0000  15.0000
1260  Default   11.000000    黃屍   11.000000  14.0000  14.0000
679   Default    7.000000   李偲嫣    7.000000  13.0000  13.0000
642   Default    6.000000    中國    6.000000  12.0000  12.0000
1497  Default    6.000000    新聞    6.000000  11.0000  11.0000
256   Default   11.000000    標準   11.000000  10.0000  10.0000
1485  Default    7.000000   黎智英    7.000000   9.0000   9.0000
1343  Default    6.000000    尊重    6.000000   8.0000   8.0000
563   Default   10.000000    學聯   10.000000   7.0000   7.0000
838   Default    5.000000    街坊    5.000000   6.0000   6.0000
1187  Default    7.000000    不同    7.000000   5.0000   5.0000
896   Default    9.000000    旺角    9.000000   4.0000   4.0000
1123  Default    9.000000    朋友    9.000000   3.0000   3.0000
343   Default   18.000000    人士   18.000000   2.0000   2.0000
9     Default    7.000000    老師    7.000000   1.0000   1.0000
...       ...         ...   ...         ...      ...      ...
1347   Topic8    1.311352    報章    2.536836   1.7606  -5.4764
1497   Topic8    3.162753    新聞    6.560111   1.6909  -4.5960
319    Topic8    2.545848    目的    5.215896   1.7032  -4.8130
642    Topic8    3.162706    中國    6.612094   1.6830  -4.5960
1278   Topic8    1.311368    心理    2.567837   1.7485  -5.4764
299    Topic8    1.311474    依家    2.592613   1.7389  -5.4763
610    Topic8    1.311407    關系    2.650800   1.7167  -5.4764
434    Topic8    2.546284    美國    5.955522   1.5708  -4.8128
563    Topic8    3.718386    學聯   10.006699   1.4305  -4.4342
124    Topic8    1.928613    議員    4.609375   1.5492  -5.0907
46     Topic8    1.311381    成員    3.234719   1.5176  -5.4764
1509   Topic8    1.311382    主觀    3.278147   1.5043  -5.4764
905    Topic8    1.311519   全世界    3.333921   1.4875  -5.4763
1471   Topic8    1.311383    公眾    3.333939   1.4874  -5.4764
441    Topic8    2.545910    政府    9.916770   1.0607  -4.8130
1562   Topic8    3.433583    政治   18.778868   0.7213  -4.5139
60     Topic8    1.311375    警員    3.913993   1.3270  -5.4764
1485   Topic8    1.928748   黎智英    7.781691   1.0256  -5.0906
691    Topic8    1.311338   周永康    4.019478   1.3004  -5.4764
671    Topic8    2.545704    清場   13.521847   0.7506  -4.8130
548    Topic8    1.311375    廣告    4.449867   1.1987  -5.4764
801    Topic8    1.311538    利益    4.659159   1.1528  -5.4763
1354   